In [1]:

!pip install -q ultralytics>=8.3.0 roboflow wandb huggingface_hub pyyaml pillow

In [2]:
from google.colab import userdata
WANDB_API_KEY = userdata.get('wandb')
HF_TOKEN = userdata.get('HF_TOKEN')
ROBOFLOW_API_KEY = userdata.get('roboflow')

In [3]:
#@title 2️⃣ Load Secrets & Login
from google.colab import userdata
import os

# Load secrets from Colab
WANDB_API_KEY = userdata.get('wandb')
HF_TOKEN = userdata.get('HF_TOKEN')
ROBOFLOW_API_KEY = userdata.get('roboflow')  # Add this to your secrets!

# Set environment variables
os.environ['WANDB_API_KEY'] = WANDB_API_KEY
os.environ['HF_TOKEN'] = HF_TOKEN

# Login to services
import wandb
wandb.login(key=WANDB_API_KEY)

from huggingface_hub import login as hf_login
hf_login(token=HF_TOKEN)

print("✅ Logged in to W&B and HuggingFace")

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: shng2025 (Imperial-College-London-SPQR) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


✅ Logged in to W&B and HuggingFace


In [4]:
#@title 3️⃣ Check GPU & Test YOLO12
import torch

print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

# Test YOLO12
from ultralytics import YOLO
test_model = YOLO("yolo12n.pt")
print("\n✅ YOLO12 loaded successfully!")

PyTorch: 2.9.0+cu126
CUDA: True
GPU: NVIDIA L4
Memory: 23.8 GB
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.

✅ YOLO12 loaded successfully!


In [5]:
#@title 4️⃣ Configuration

# =============================================================================
# EDIT THESE SETTINGS
# =============================================================================

# Model: yolo12n (fast) | yolo12s | yolo12m (balanced) | yolo12l | yolo12x (accurate)
MODEL = "yolo12m.pt"

# Training
EPOCHS = 1
BATCH_SIZE = 16  # Reduce to 8 if OOM
IMG_SIZE = 640

# Dataset (Roboflow)
WORKSPACE = "shahed136"
PROJECT = "shahed136-detect"
VERSION = 1

# Logging
WANDB_ENTITY = "Imperial-College-London-SPQR"
WANDB_PROJECT = "European-Defense-Hackathon-Warsaw"
HF_REPO = "shng2025/EDTH-Warsaw-shahed136-detector"

print(f"""
Configuration:
  Model: {MODEL}
  Epochs: {EPOCHS}
  Batch: {BATCH_SIZE}
  Image Size: {IMG_SIZE}
  W&B: {WANDB_ENTITY}/{WANDB_PROJECT}
  HuggingFace: {HF_REPO}
""")


Configuration:
  Model: yolo12m.pt
  Epochs: 1
  Batch: 16
  Image Size: 640
  W&B: Imperial-College-London-SPQR/European-Defense-Hackathon-Warsaw
  HuggingFace: shng2025/EDTH-Warsaw-shahed136-detector



In [6]:
#@title 5️⃣ Download Dataset
from roboflow import Roboflow
import yaml

rf = Roboflow(api_key=ROBOFLOW_API_KEY)  # Already loaded from secrets
project = rf.workspace("edthwarsaw").project("shahed136-detect-emoo1")
version = project.version(1)
dataset = version.download("yolov8")

DATASET_PATH = dataset.location
DATA_YAML = f"{DATASET_PATH}/data.yaml"

with open(DATA_YAML, 'r') as f:
    data_info = yaml.safe_load(f)
CLASS_NAMES = data_info.get("names", [])
if isinstance(CLASS_NAMES, dict):
    CLASS_NAMES = [CLASS_NAMES[i] for i in sorted(CLASS_NAMES.keys())]

print(f"✅ Dataset: {DATASET_PATH}")
print(f"Classes: {CLASS_NAMES}")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to shahed136-detect-1 in yolov8:: 100%|██████████| 16246/16246 [00:01<00:00, 10082.34it/s]


✅ Dataset: /content/shahed136-detect-1
Classes: ['1']


In [7]:
#@title 6️⃣ Setup Logging Classes
import json
import random
from pathlib import Path
from datetime import datetime
from PIL import Image
from ultralytics import YOLO


class WandBLogger:
    """W&B logger with visual predictions."""

    def __init__(self, run_name, config):
        import wandb
        self.wandb = wandb
        self.run = wandb.init(
            entity=WANDB_ENTITY,
            project=WANDB_PROJECT,
            name=run_name,
            tags=["yolo12", "drone-detection", "shahed-136", "defense"],
            config=config,
        )
        self.run_id = self.run.id
        print(f"[W&B] {self.run.url}")

    def log(self, metrics, step=None):
        self.wandb.log(metrics, step=step)

    def log_predictions(self, model_path, images, class_names, epoch):
        """Log detection visualizations."""
        # Load model fresh to use YOLO wrapper's predict
        temp_model = YOLO(model_path)
        results = temp_model.predict(images, conf=0.25, verbose=False)
        wandb_images = []

        for img_path, result in zip(images, results):
            img = Image.open(img_path)
            boxes_data = []

            if result.boxes is not None:
                for box in result.boxes:
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                    conf = float(box.conf[0])
                    cls_id = int(box.cls[0])
                    cls_name = class_names[cls_id] if cls_id < len(class_names) else str(cls_id)
                    boxes_data.append({
                        "position": {
                            "minX": float(x1)/img.width, "minY": float(y1)/img.height,
                            "maxX": float(x2)/img.width, "maxY": float(y2)/img.height,
                        },
                        "class_id": cls_id,
                        "box_caption": f"{cls_name}: {conf:.2f}",
                        "scores": {"confidence": conf},
                    })

            wandb_images.append(self.wandb.Image(
                img,
                boxes={"predictions": {
                    "box_data": boxes_data,
                    "class_labels": {i: n for i, n in enumerate(class_names)},
                }},
                caption=f"Epoch {epoch}",
            ))

        self.wandb.log({"predictions/samples": wandb_images}, step=epoch)

    def log_gt_vs_pred(self, model_path, img_paths, label_paths, class_names, epoch):
        """Log ground truth vs predictions."""
        temp_model = YOLO(model_path)
        images = []
        for img_path, label_path in zip(img_paths, label_paths):
            img = Image.open(img_path)
            w, h = img.size

            # Ground truth
            gt_boxes = []
            if Path(label_path).exists():
                with open(label_path) as f:
                    for line in f:
                        parts = line.strip().split()
                        if len(parts) >= 5:
                            cls_id = int(parts[0])
                            xc, yc, bw, bh = map(float, parts[1:5])
                            gt_boxes.append({
                                "position": {"minX": xc-bw/2, "minY": yc-bh/2, "maxX": xc+bw/2, "maxY": yc+bh/2},
                                "class_id": cls_id,
                                "box_caption": f"GT: {class_names[cls_id] if cls_id < len(class_names) else cls_id}",
                            })

            # Predictions
            result = temp_model.predict(img_path, conf=0.25, verbose=False)[0]
            pred_boxes = []
            if result.boxes is not None:
                for box in result.boxes:
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                    conf = float(box.conf[0])
                    cls_id = int(box.cls[0])
                    pred_boxes.append({
                        "position": {"minX": x1/w, "minY": y1/h, "maxX": x2/w, "maxY": y2/h},
                        "class_id": cls_id,
                        "box_caption": f"Pred: {class_names[cls_id] if cls_id < len(class_names) else cls_id} ({conf:.2f})",
                        "scores": {"confidence": conf},
                    })

            labels = {i: n for i, n in enumerate(class_names)}
            images.append(self.wandb.Image(img, boxes={
                "ground_truth": {"box_data": gt_boxes, "class_labels": labels},
                "predictions": {"box_data": pred_boxes, "class_labels": labels},
            }))

        self.wandb.log({"labeling_quality/comparison": images}, step=epoch)

    def log_label_distribution(self, dataset_path, class_names):
        """Log class distribution."""
        counts = {}
        for split in ["train", "valid", "val", "test"]:
            labels_dir = Path(dataset_path) / split / "labels"
            if not labels_dir.exists():
                continue
            for f in labels_dir.glob("*.txt"):
                for line in open(f):
                    parts = line.strip().split()
                    if parts:
                        cls_id = int(parts[0])
                        name = class_names[cls_id] if cls_id < len(class_names) else str(cls_id)
                        counts[name] = counts.get(name, 0) + 1

        table = self.wandb.Table(data=[[k, v] for k, v in counts.items()], columns=["class", "count"])
        self.wandb.log({"dataset/label_distribution": self.wandb.plot.bar(table, "class", "count", title="Labels")})
        print(f"[W&B] Label counts: {counts}")

    def finish(self):
        self.wandb.finish()


class HFCheckpointer:
    """HuggingFace Hub checkpointing."""

    def __init__(self, run_name):
        from huggingface_hub import HfApi, create_repo
        self.api = HfApi()
        self.run_name = run_name
        self.run_folder = f"runs/{run_name}"

        try:
            create_repo(HF_REPO, private=True, exist_ok=True, repo_type="model")
            print(f"[HF] https://huggingface.co/{HF_REPO}")
        except Exception as e:
            print(f"[HF] {e}")

    def upload(self, model_path, epoch, metrics=None, is_best=False):
        from huggingface_hub import upload_file

        if not Path(model_path).exists():
            return

        filename = f"{self.run_folder}/best.pt" if is_best else f"{self.run_folder}/epoch_{epoch:04d}.pt"

        try:
            upload_file(model_path, filename, HF_REPO, commit_message=f"Epoch {epoch}" + (" (best)" if is_best else ""))
            print(f"[HF] Uploaded: {filename}")
        except Exception as e:
            print(f"[HF] Upload failed: {e}")


print("✅ Logging classes ready")

✅ Logging classes ready


In [ ]:
#@title 7️⃣ 🚀 Train YOLO12
from ultralytics import YOLO
from ultralytics.utils import SETTINGS

SETTINGS["wandb"] = True

# Generate run name
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model_name = Path(MODEL).stem
run_name = f"{model_name}_{timestamp}"
print(f"Run: {run_name}")

# Initialize loggers
config = {"model": MODEL, "epochs": EPOCHS, "batch": BATCH_SIZE, "img_size": IMG_SIZE}
wandb_logger = WandBLogger(run_name, config)
hf_checkpointer = HFCheckpointer(run_name)

# Log dataset stats
wandb_logger.log_label_distribution(DATASET_PATH, CLASS_NAMES)

# Get sample images for visualization
val_dir = Path(DATASET_PATH) / "valid" / "images"
if not val_dir.exists():
    val_dir = Path(DATASET_PATH) / "val" / "images"

sample_images = [str(p) for p in list(val_dir.glob("*.jpg"))[:16]]

# Get GT-pred pairs
labels_dir = val_dir.parent / "labels"
pairs = []
for img in val_dir.glob("*"):
    if img.suffix.lower() in [".jpg", ".jpeg", ".png"]:
        lbl = labels_dir / f"{img.stem}.txt"
        if lbl.exists():
            pairs.append((str(img), str(lbl)))
pairs = pairs[:8]

# Load model
model = YOLO(MODEL)

# Callbacks
best_map = [0.0]

def on_epoch_end(trainer):
    epoch = trainer.epoch
    metrics = trainer.metrics

    # Log metrics
    wandb_metrics = {
        "train/box_loss": metrics.get("train/box_loss", 0),
        "train/cls_loss": metrics.get("train/cls_loss", 0),
        "metrics/mAP50": metrics.get("metrics/mAP50(B)", 0),
        "metrics/mAP50-95": metrics.get("metrics/mAP50-95(B)", 0),
        "metrics/precision": metrics.get("metrics/precision(B)", 0),
        "metrics/recall": metrics.get("metrics/recall(B)", 0),
    }
    wandb_logger.log(wandb_metrics, step=epoch)

    # Visual predictions every 5 epochs
    if epoch % 5 == 0 and sample_images:
        # Use last.pt for predictions
        weights_path = Path(trainer.save_dir) / "weights" / "last.pt"
        if weights_path.exists():
            wandb_logger.log_predictions(str(weights_path), sample_images, CLASS_NAMES, epoch)
            if pairs:
                wandb_logger.log_gt_vs_pred(str(weights_path), [p[0] for p in pairs], [p[1] for p in pairs], CLASS_NAMES, epoch)

    # HuggingFace checkpoints
    current_map = metrics.get("metrics/mAP50(B)", 0)
    is_best = current_map > best_map[0]
    if is_best:
        best_map[0] = current_map

    if epoch % 10 == 0 or is_best:
        save_dir = trainer.save_dir
        last_pt = Path(save_dir) / "weights" / "last.pt"
        best_pt = Path(save_dir) / "weights" / "best.pt"

        if last_pt.exists():
            hf_checkpointer.upload(str(last_pt), epoch)
        if is_best and best_pt.exists():
            hf_checkpointer.upload(str(best_pt), epoch, is_best=True)

model.add_callback("on_train_epoch_end", on_epoch_end)

# TRAIN!
results = model.train(
    data=DATA_YAML,
    epochs=EPOCHS,
    batch=BATCH_SIZE,
    imgsz=IMG_SIZE,
    patience=20,

    optimizer="AdamW",
    lr0=0.001,
    lrf=0.01,
    weight_decay=0.0005,

    augment=True,
    mosaic=1.0,

    project="runs/detect",
    name=run_name,
    exist_ok=True,
    save_period=10,

    device=0,
    workers=4,
    amp=True,

    plots=True,
    save=True,
    val=True,
    verbose=True,
)

wandb_logger.finish()

print(f"\n" + "="*60)
print("✅ TRAINING COMPLETE!")
print(f"Results: runs/detect/{run_name}")
print(f"Best weights: runs/detect/{run_name}/weights/best.pt")
print(f"W&B: https://wandb.ai/{WANDB_ENTITY}/{WANDB_PROJECT}")
print(f"HuggingFace: https://huggingface.co/{HF_REPO}")

Run: yolo12m_20251206_144537


[W&B] https://wandb.ai/Imperial-College-London-SPQR/European-Defense-Hackathon-Warsaw/runs/rdddd6u2
[HF] https://huggingface.co/shng2025/EDTH-Warsaw-shahed136-detector
[W&B] Label counts: {'1': 8117}
Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/shahed136-detect-1/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, mod

In [ ]:
#@title 8️⃣ Final Validation
val_metrics = model.val(data=DATA_YAML)

print(f"\n📊 Final Results:")
print(f"  mAP50:     {val_metrics.box.map50:.4f}")
print(f"  mAP50-95:  {val_metrics.box.map:.4f}")
print(f"  Precision: {val_metrics.box.mp:.4f}")
print(f"  Recall:    {val_metrics.box.mr:.4f}")

In [ ]:
#@title 9️⃣ Test on Image
from google.colab import files
from IPython.display import display, Image as IPImage

# Upload test image
print("Upload an image to test:")
uploaded = files.upload()

# Load best model
best_model = YOLO(f"runs/detect/{run_name}/weights/best.pt")

for filename in uploaded.keys():
    results = best_model(filename, save=True)
    print(f"\nDetections in {filename}:")
    for box in results[0].boxes:
        cls_id = int(box.cls[0])
        conf = float(box.conf[0])
        cls_name = CLASS_NAMES[cls_id] if cls_id < len(CLASS_NAMES) else str(cls_id)
        print(f"  - {cls_name}: {conf:.1%}")

    # Show result
    result_path = list(Path("runs/detect/predict").glob(f"*{Path(filename).stem}*"))[0]
    display(IPImage(filename=str(result_path)))

In [ ]:
#@title 🔟 Download Model
from google.colab import files

# Download best weights
files.download(f"runs/detect/{run_name}/weights/best.pt")

# Optional: Export to ONNX and download
# model.export(format="onnx")
# files.download(f"runs/detect/{run_name}/weights/best.onnx")